# Scraping de la page Wikipedia des oiseaux français

Ce programme scrape la page Wikipedia https://en.wikipedia.org/wiki/List_of_birds_of_Metropolitan_France pour récupérer la liste des oiseaux français avec leurs informations :
- la catégorie vernaculaire
- l'ordre
- la famille
- le nom anglais
- le nomm latin
- le nom français (si fourni)
- l'URL du lien Wikipedia anglais
- l'URL du lein Wikipedia français (si fourni)

Le programme créé un fichier csv nommé 'birds_output.csv' contenant une ligne par espèce avec les information décrites ci-dessus

In [1]:
# Installation des pacakages
! pip install requests
! pip install beautifulsoup4


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Imports
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime

In [3]:
# Récupération du code HTML de la page Wikipedia
path_wiki = 'https://en.wikipedia.org/wiki/List_of_birds_of_Metropolitan_France'
response = requests.get(path_wiki)
print("reponse serveur = " , response.status_code)
print(response.content[:500])

reponse serveur =  200
b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clien'


In [4]:
# Récupération de la liste des catégories d'oiseaux
# Les catégories sont dans les balises <h2>
html = response.text
soup = BeautifulSoup(html)
h2_List = list(soup.find_all("h2"))

# La première balise <h2> ne nous intéresse pas
del h2_List[0] 

In [5]:

# Initialisation de la liste des oiseaux, qui sera complétée à chaque espèce scrapée
birds_list = []

# On boucle sur les catégories = tags <h2>. Les 3 dernières lignes ne nous intéressent pas
for cat in range(0,len(h2_List)-3) :
    
  pointeur = h2_List[cat]

  # On récupère l'ordre et la famille
  pointeur_inside = pointeur.find_next_sibling("p")
  pointeur_inside = pointeur_inside.find("a")
  order = pointeur_inside.text
  pointeur_inside = pointeur_inside.find_next_sibling("a")
  familly = pointeur_inside.text

  pointeur = pointeur.find_next('ul')
  liste_li = pointeur.find_all('li')
    
  # On boucle sur la liste des espèces
  for index in range(len(liste_li)) : 
    # Pour chaque espèce on récupère les noms en anglais, latin et français, et les URLs fr et en
    pointeur_li = liste_li[index]
    pointeur_li = pointeur_li.find("a")
    nom_en = pointeur_li.text
    url_en = 'https://en.wikipedia.org' + pointeur_li['href']
    pointeur_li = pointeur_li.find_next_sibling("i")
    nom_lt = pointeur_li.text
    pointeur_li = pointeur_li.find_next_sibling("i")

    # On teste que le nom et l'url français sont présents
    if pointeur_li :
      url_fr = pointeur_li.find('a').get("href")
      nom_fr = pointeur_li.text
    else :
      nom_fr = ""
      url_fr = ""

    # Construction de la nouvelle ligne + ajout au dataframe
    cat_name = h2_List[cat].text
    cat_name = cat_name.replace("[edit]", "")
    birds_list.append({'Categorie': cat_name,'Ordre' : order, 'Famille': familly, 'Nom_EN': nom_en, 'Nom_LT' : nom_lt, 'Nom_FR' : nom_fr, 'Url_EN' : url_en, 'Url_FR' : url_fr})

In [6]:
# On convertit la liste en datframe
birds_df = pd.DataFrame(birds_list)
birds_df

,Categorie,Ordre,Famille,Nom_EN,Nom_LT,Nom_FR,Url_EN,Url_FR
0,"Ducks, geese, and waterfowl",Anseriformes,Anatidae,Snow goose,Anser caerulescens,oie des neiges,https://en.wikipedia.org/wiki/Snow_goose,https://fr.wikipedia.org/wiki/Oie_des_neiges
1,"Ducks, geese, and waterfowl",Anseriformes,Anatidae,Graylag goose,Anser anser,oie cendrée,https://en.wikipedia.org/wiki/Graylag_goose,https://fr.wikipedia.org/wiki/Oie_cendr%C3%A9e
2,"Ducks, geese, and waterfowl",Anseriformes,Anatidae,Greater white-fronted goose,Anser albifrons,oie rieuse,https://en.wikipedia.org/wiki/Greater_white-fr...,https://fr.wikipedia.org/wiki/Oie_rieuse
3,"Ducks, geese, and waterfowl",Anseriformes,Anatidae,Lesser white-fronted goose,Anser erythropus,oie naine,https://en.wikipedia.org/wiki/Lesser_white-fro...,https://fr.wikipedia.org/wiki/Oie_naine
4,"Ducks, geese, and waterfowl",Anseriformes,Anatidae,Taiga bean-goose,Anser fabalis,oie des moissons,https://en.wikipedia.org/wiki/Taiga_bean-goose,https://fr.wikipedia.org/wiki/Oie_des_moissons
...,...,...,...,...,...,...,...,...
617,New World warblers,Passeriformes,Parulidae,Yellow warbler,Setophaga petechia,paruline jaune,https://en.wikipedia.org/wiki/Yellow_warbler,https://fr.wikipedia.org/wiki/Paruline_jaune
618,New World warblers,Passeriformes,Parulidae,Chestnut-sided warbler,Setophaga pensylvanica,paruline à flancs marron,https://en.wikipedia.org/wiki/Chestnut-sided_w...,https://fr.wikipedia.org/wiki/Paruline_%C3%A0_...
619,New World warblers,Passeriformes,Parulidae,Blackpoll warbler,Setophaga striata,paruline rayée,https://en.wikipedia.org/wiki/Blackpoll_warbler,https://fr.wikipedia.org/wiki/Paruline_ray%C3%A9e
620,Cardinals and allies,Passeriformes,Cardinalidae,Scarlet tanager,Piranga olivacea,tangara écarlate,https://en.wikipedia.org/wiki/Scarlet_tanager,https://fr.wikipedia.org/wiki/Tangara_%C3%A9ca...


In [7]:
for column in birds_df.columns:
    print(column, birds_df[column].duplicated().sum())

Categorie 539
Ordre 598
Famille 539
Nom_EN 0
Nom_LT 0
Nom_FR 5
Url_EN 0
Url_FR 6


In [8]:
birds_df.isna().sum()

Categorie    0
Ordre        0
Famille      0
Nom_EN       0
Nom_LT       0
Nom_FR       0
Url_EN       0
Url_FR       0
dtype: int64

In [9]:
# On exporte le dataframe dans un fichier csv
date_courante = datetime.datetime.now()
date_formatee = date_courante.strftime('%y%m%d_%H%M%S')
file_name = 'french_bird_wiki_' + str(date_formatee) +'.csv'
birds_df.to_csv(file_name, index=False, sep= ";")